In [4]:
import pymongo
import numpy as np
import pandas as pd

In [2]:
# !pip3 install pymongo

In [5]:
host_ip = 'localhost'
client = pymongo.MongoClient(host_ip, 27017)

db  = client['hotel_zh_gaodeng']
col0 = db["yws_hotel_mapping"]
col1 = db["yws_hotel_mapping_1"]
col2 = db["yws_hotel_mapping_2"]
df0 = pd.DataFrame(list(col0.find()))
df1 = pd.DataFrame(list(col1.find()))
df2 = pd.DataFrame(list(col2.find()))
df = pd.concat([df0, df1, df2])
# df

In [6]:
df = df.dropna(subset=['brand_name'])
# df

In [7]:
# random_state = 1
# df = df.sample(frac=1,random_state=random_state).reset_index(drop=True)
# df

In [19]:
# df[df['name']=='亚朵']

In [19]:
def remove_sths(s, sths):
    for sth in sths:
        s = s.replace(sth, "")
    return s
df["name"] = df["brand_name"].apply(lambda x:remove_sths(x,sths=["酒店", '饭店']))
# df

In [8]:
def has_sth(s, sth):
    if sth in s:
        return 1
    else:
        return 0
df["has_sth"] = df["hotel_name"].apply(lambda x:has_sth(x,sth="酒店"))

In [24]:
def has_sths(s, sths):
    ss = []
    for sth in sths:
        if sth in s:
            ss.append(sth)
    return ss
df["has_addr"] = df["hotel_name"].apply(lambda x:has_sths(x,sths=set(addr_names)))
df

,_id,id,hotel_name,city_id,brand_id,brand_name,address,latitude,longitude,contact_tel,supply,hotel_code,是否匹配,has_sth,name,has_addr
0,60d2e3ef120a0939fd592bca,346.0,北京建国饭店,1,54.0,建国饭店,建外大街5号,39.909191,116.452753,+86-10-65002233,NaN,NaN,NaN,0,建国,[北京]
1,60d2e3ef120a0939fd592bcb,380.0,北京京伦饭店,1,168.0,京伦饭店,建国门外大街3号,39.909004,116.454721,010-65002266,NaN,NaN,NaN,0,京伦,[北京]
2,60d2e3ef120a0939fd592bcc,392.0,如家精选酒店(北京广渠门内地铁站鸿润店),1,170.0,如家精选,广渠门内白桥大街28号,39.894815,116.440098,010-67196888-9,NaN,NaN,NaN,1,如家精选,[北京]
3,60d2e3ef120a0939fd592bcd,454.0,如家酒店(广饶乐安大街县政府店),977,22.0,如家,乐安大街506号,37.053276,118.405346,-2116937904,NaN,NaN,NaN,1,如家,[]
4,60d2e3ef120a0939fd592bce,480.0,如家酒店(广饶大王镇政府店),977,22.0,如家,大王镇派出所东临,36.984996,118.527638,+86-546-6523333-9,NaN,NaN,NaN,1,如家,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6593,60d309da120a0939fd5ab4ae,NaN,麗枫酒店（建德新安江店）,15,NaN,麗枫,新安江街道沧滩路2号,29.471807,119.272795,0571-64198666,B,1660443400,0.0,1,麗枫,[]
6594,60d309da120a0939fd5ab4af,NaN,凯里亚德酒店（邵阳九盛北海店）,509,NaN,凯里亚德,西湖北路与崀山路交汇处步步高超市旁,27.248914,111.456288,0739-5205555,B,1546524613,0.0,1,凯里亚德,[]
6595,60d309da120a0939fd5ab4b0,NaN,城市便捷酒店（丽江古城七星街店）,32,NaN,城市便捷酒店,西安街道雪山路寨后上村104号,26.875920,100.217880,0888-6999666,B,1617836177,0.0,1,城市便捷,[]
6596,60d309da120a0939fd5ab4b1,NaN,白玉兰长沙东风路省博物馆酒店,128,NaN,白玉兰酒店,东风路1号,28.203858,112.990377,0731-88281888-0/15116488430,B,1584565632,0.0,1,白玉兰,[]


In [73]:
# df[df['name']=='格林豪泰']

In [74]:
# len(max(df["hotel_name"].to_list(), key=len))  # 酒店名称的最大长度

In [30]:
# sorted(df.groupby('name').size())
# dd_df = df.drop_duplicates(['name'], keep='first',inplace=False)
# df = dd_df

In [31]:
len(df) * 0.8

26876.0

In [81]:
[chunk_tags[0]] * 10

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [25]:
def writ2file(tags, hotel_name, fw=None):
    for i in range(len(hotel_name)):
        if fw:
            fw.write("{} {}\n".format(hotel_name[i], tags[i]))
    if fw:
        fw.writelines("\n")

In [26]:
def generate_tags(hotel_name, loc_inds, brd_inds):
    # 生成tags
    tags = [chunk_tags[0]] * len(hotel_name)
    for inds in loc_inds:
        for i in range(inds[0], inds[1]):
            if i == inds[0]:
                tags[i] = chunk_tags[3]
            else:
                tags[i] = chunk_tags[4]
    for inds in brd_inds:
        for i in range(inds[0], inds[1]):
            if i == inds[0]:
                tags[i] = chunk_tags[1]
            else:
                tags[i] = chunk_tags[2]
    return tags

def data_aug(brd_inds, hotel_name):
    # 在hotel_name前面随机增加地名
    s_ind, e_ind = brd_inds[0]
    brd_inds = []
    loc_ids = []
    adds = addr_names[random.randint(0,len(addr_names)-1)]
    s_ind += len(adds)
    e_ind += len(adds)
    hotel_name = adds + hotel_name
    loc_s_inds = 0
    loc_e_inds = len(adds)
    loc_inds.append((0,len(adds)))
    brd_inds.append((s_ind, e_ind))
    return brd_inds, loc_ids, hotel_name


def preprocess(hotel_name, name):
    # 预处理
    hotel_name = hotel_name.replace(" ", "")
    name = name.replace(" ", "")
    loc_inds = []
    brd_inds = []
    flag = False
    if name in hotel_name:
        flag = True
        s_ind = hotel_name.index(name)
        e_ind = s_ind + len(name)
        if has_sth == 1:
            h_ind = hotel_name.index("酒店")
            if h_ind <= s_ind + len(name) + 2:
                # e_ind = h_ind
                e_ind = h_ind + 2
        brd_inds.append((s_ind, e_ind))
    return brd_inds, loc_inds, hotel_name, flag

In [27]:
def process_addr(hotel_name, addrs):
    loc_inds = []
    for name in addrs:
        s_ind = hotel_name.index(name)
        e_ind = s_ind + len(name)
        loc_inds.append((s_ind, e_ind))
    return loc_inds

In [29]:
import random
chunk_tags = ['O', 'B-BRD', 'I-BRD', 'B-LOC', 'I-LOC']
fw_train = open('data/hotel_train_data.data', 'w')
fw_test = open('data/hotel_test_data.data', 'w')

for idx in range(len(df)):
# for idx in range(100,200):
    if idx < len(df) * 0.8:
        fw = fw_train
    else:
        fw = fw_test
    name = df.iloc[idx]['name']
    hotel_name = df.iloc[idx]['hotel_name']
    has_sth = df.iloc[idx]['has_sth']
    addrs = df.iloc[idx]['has_addr']
    brd_inds, loc_inds, hotel_name, flag = preprocess(hotel_name, name)
    if flag:
        # loc_inds += process_addr(hotel_name, addrs)
        tags = generate_tags(hotel_name, loc_inds, brd_inds)
        writ2file(tags, hotel_name, fw=fw)
        # aug
        # brd_inds, loc_ids, hotel_name = data_aug(brd_inds, hotel_name)
        # # loc_inds += process_addr(hotel_name, addrs)
        # tags = generate_tags(hotel_name, loc_inds, brd_inds)
        # writ2file(tags, hotel_name, fw=fw)

fw_train.close()
fw_test.close()

In [135]:
str_1='wo shi yi zhi da da niu '
char_1='yi'
nPos=str_1.index(char_1)
print(nPos)

7


In [20]:
# 获取中国所有的地理名称
adm_df = pd.read_excel("city_list_modified.xlsx")
# adm_df["把“区”去掉后"]
adm_df = adm_df.dropna(subset=['name'])
# addr_names = adm_df['name'].to_list()

def get_level(s):
    return s % 10000 == 0 
adm_df["level"] = adm_df["行政区划代码"].apply(lambda x:get_level(x))

adm_df

,行政区划代码,单位名称,把“市”去掉了,拼音,把“区”去掉后,name,Unnamed: 6,Unnamed: 7,level
0,110000,北京市,北京,BeiJing,北京,北京,NaN,只到区县一级，没有镇村信息,True
1,110101,东城区,东城区,DongChengQu,东城,东城,NaN,注：无港澳台地区、海南三沙市下属区县信息,False
2,110102,西城区,西城区,XiChengQu,西城,西城,NaN,NaN,False
3,110105,朝阳区,朝阳区,ChaoYangQu,朝阳,朝阳,NaN,NaN,False
4,110106,丰台区,丰台区,FengTaiQu,丰台,丰台,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...
3205,659010,胡杨河市,胡杨河,HuYangHe,胡杨河,胡杨河,NaN,NaN,False
3206,710000,台湾省,台湾省,TaiWanSheng,台湾省,台湾,NaN,NaN,True
3618,820000,澳门特别行政区,澳门特别行政区,AoMenTeBieXingZhengQu,澳门特别行政,澳门,NaN,NaN,True
3629,990000,南沙区,南沙区,NanShaQu,南沙,南沙,NaN,NaN,True


In [21]:
adm_df = adm_df[adm_df["level"]==True]
adm_df

,行政区划代码,单位名称,把“市”去掉了,拼音,把“区”去掉后,name,Unnamed: 6,Unnamed: 7,level
0,110000,北京市,北京,BeiJing,北京,北京,NaN,只到区县一级，没有镇村信息,True
17,120000,天津市,天津,TianJin,天津,天津,NaN,NaN,True
34,130000,河北省,河北省,HeBeiSheng,河北省,河北省,NaN,NaN,True
213,140000,山西省,山西省,ShanXiSheng,山西省,山西省,NaN,NaN,True
342,150000,内蒙古自治区,内蒙古自治区,NeiMengGuZiZhiQu,内蒙古自治,内蒙古自治,NaN,NaN,True
458,210000,辽宁省,辽宁省,LiaoNingSheng,辽宁省,辽宁省,NaN,NaN,True
573,220000,吉林省,吉林省,JiLinSheng,吉林省,吉林省,NaN,NaN,True
643,230000,黑龙江省,黑龙江省,HeiLongJiangSheng,黑龙江省,黑龙江省,NaN,NaN,True
778,310000,上海市,上海,ShangHai,上海,上海,NaN,NaN,True
795,320000,江苏省,江苏省,JiangSuSheng,江苏省,江苏省,NaN,NaN,True


In [22]:
# "张家港" in addr_names

adm_df["name"] = adm_df["name"].apply(lambda x:remove_sths(x, sths=["省", '自治','壮族','回族','维吾尔']))
addr_names = adm_df['name'].to_list()
addr_names

<ipython-input-22-17ed31cda920>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adm_df["name"] = adm_df["name"].apply(lambda x:remove_sths(x, sths=["省", '自治','壮族','回族','维吾尔']))


['北京',
 '天津',
 '河北',
 '山西',
 '内蒙古',
 '辽宁',
 '吉林',
 '黑龙江',
 '上海',
 '江苏',
 '浙江',
 '安徽',
 '福建',
 '江西',
 '山东',
 '河南',
 '湖北',
 '湖南',
 '广东',
 '广西',
 '海南',
 '重庆',
 '四川',
 '贵州',
 '云南',
 '西藏',
 '陕西',
 '甘肃',
 '青海',
 '宁夏',
 '新疆',
 '台湾',
 '澳门',
 '南沙',
 '西沙']

In [23]:
# len(addr_names)

In [142]:
client = pymongo.MongoClient('localhost',27017)

db  = client['hotel_zh_gaodeng']
col = db["gaodeng_siamese_result_v20210607_checked"]
df_check = pd.DataFrame(list(col.find()))
df_check.head()

,_id,supply,supplier_hotel_id,supplier_hotel_name,supplier_brand_name,supplier_city_id,supplier_contact_tel,supplier_latitude,supplier_longitude,supplier_address,...,distributor_brand_name,distributor_city_id,distributor_contact_tel,distributor_latitude,distributor_longitude,distributor_address,score,confidence,rule_check,human
0,60d5451a301f052849d780b1,A,10002,如家酒店·neo(北京前门步行街店),如家,1,010-63173366转9,39.893163,116.397515,北京市西城区前门粮食店街61号,...,如家,1,+86-10-63173366,39.893216,116.397570,粮食店街61号,1.0,high,1,1
1,60d5451a301f052849d780b2,A,10011,如家酒店·neo(北京朝阳公园朝阳火车站店),如家,1,010-85833388转9,39.935185,116.503187,北京市朝阳区石佛营东里105号院北侧,...,如家,1,+86-10-85833388,39.935211,116.502932,石佛营东里小区105号,1.0,high,1,1
2,60d5451a301f052849d780b3,A,10014,如家酒店(北京小西天店),如家,1,010-62231199转9,39.951059,116.362893,海淀区新街口外北大街文慧园斜街6号,...,如家,1,010-62231199,39.951486,116.362582,文慧园斜街6号,1.0,high,1,1
3,60d5451a301f052849d780b4,A,10016,如家酒店(北京大观园店),如家,1,010-83559989转9,39.875206,116.353624,北京市西城区南菜园街9号院6号楼国泰大厦院内,...,如家,1,010-83559989,39.875151,116.352878,南菜园街9号院6号楼北侧,1.0,high,1,1
4,60d5451a301f052849d780b5,A,10017,如家酒店(北京东直门店),如家,1,010-51203288转9,39.940288,116.440442,北京市东城区东直门外大街新中街甲2号,...,如家,1,010-51203288,39.940737,116.439956,东直门外大街新中街甲2号,1.0,high,1,1


In [156]:
tmp_df = df_check[(df_check['supplier_hotel_name'] != df_check['distributor_hotel_name']) & (df_check['human']==1)][['supplier_hotel_name', 'distributor_hotel_name','supplier_contact_tel', 'distributor_contact_tel']]

In [159]:
supplier_hotel_names = tmp_df["supplier_hotel_name"].to_list()
distributor_hotel_names = tmp_df['distributor_hotel_name'].to_list()

In [160]:
# 保存data-set-np
import pickle  
# aa = Person("JGood", 2)  
# aa.show()  
a = [supplier_hotel_names, distributor_hotel_names]
f=open('./data/check_data.pkl','wb')  
pickle.dump(a,f,0) 

In [173]:
cases = ['如家酒店(杭州四季青服装市场清江路店)','清沐酒店(杭州四季青服装市场清江路店)', '澳门新葡京酒店', '澳门葡京酒店', '东莞长安海悦花园大酒店', '桔子水晶深圳宝安机场碧海湾酒店', 
        '博鳌和悦景澜海景度假酒店','上海虹桥国家会展中心希尔顿欢朋酒店', '厦门磐基希尔顿酒店','徐州复兴苏宁广场苏宁雅悦酒店','古北水镇新青年艺术酒店','古北水镇古北之光温泉酒店','博乐']
import addressparser

# for case in cases:
df = addressparser.transform(cases)
print(df)

           省          市    区                   地名
0        浙江省        杭州市       如家酒店(杭州四季青服装市场清江路店)
1        浙江省        杭州市       清沐酒店(杭州四季青服装市场清江路店)
2    澳门特别行政区    澳门特别行政区                     新葡京酒店
3    澳门特别行政区    澳门特别行政区                      葡京酒店
4        广东省        东莞市                 长安海悦花园大酒店
5        广东省        深圳市           桔子水晶深圳宝安机场碧海湾酒店
6                                    博鳌和悦景澜海景度假酒店
7        上海市        上海市           虹桥国家会展中心希尔顿欢朋酒店
8        福建省        厦门市                   磐基希尔顿酒店
9        江苏省        徐州市              复兴苏宁广场苏宁雅悦酒店
10                                    古北水镇新青年艺术酒店
11  新疆维吾尔自治区  博尔塔拉蒙古自治州  温泉县         古北水镇古北之光温泉酒店
12  新疆维吾尔自治区  博尔塔拉蒙古自治州  博乐市                     
